In [1]:
import collections

def adjacence(i):
    adj = {
        0: [1, 3], 1: [0, 2, 4], 2: [1, 5],
        3: [0, 4, 6], 4: [1, 3, 5, 7], 5: [2, 4, 8],
        6: [3, 7], 7: [4, 6, 8], 8: [5, 7]
    }
    return adj[i]

board_resolu = (1, 2, 3, 4, 5, 6, 7, 8, 0) # Utilisation de tuples

def solve_taquin(init):
    init = tuple(init) # On s'assure que c'est un tuple
    # distance[state] = distance à l'origine
    # origine[state] = l'état précédent (parent)
    distance = {init: 0}
    origine = {init: None}
    
    # File pour le BFS (indispensable pour Dijkstra sur graphe non pondéré)
    queue = collections.deque([init])
    
    while queue:
        current_board = queue.popleft() # On prend le premier (BFS)
        
        if current_board == board_resolu:
            return origine # On a trouvé le chemin
            
        vide = current_board.index(0)
        
        for move in adjacence(vide):
            # On crée une copie pour ne pas modifier l'original
            temp_list = list(current_board)
            temp_list[vide], temp_list[move] = temp_list[move], temp_list[vide]
            next_board = tuple(temp_list)
            
            if next_board not in distance:
                distance[next_board] = distance[current_board] + 1
                origine[next_board] = current_board
                queue.append(next_board)
                
    return None

def find_sequence(board_init):
    origine = solve_taquin(board_init)
    if not origine: return "Pas de solution"
    
    chemin = []
    etape = board_resolu
    while etape is not None:
        chemin.append(list(etape))
        etape = origine[etape]
    
    return chemin[::-1] # On inverse pour avoir l'ordre départ -> fin

# Test
init = [1, 6, 2, 7, 8, 0, 4, 5, 3]
sequence = find_sequence(init)
print(f"Résolu en {len(sequence)-1} étapes.")
for s in sequence:
    print(s)

Résolu en 21 étapes.
[1, 6, 2, 7, 8, 0, 4, 5, 3]
[1, 6, 2, 7, 0, 8, 4, 5, 3]
[1, 6, 2, 0, 7, 8, 4, 5, 3]
[1, 6, 2, 4, 7, 8, 0, 5, 3]
[1, 6, 2, 4, 7, 8, 5, 0, 3]
[1, 6, 2, 4, 0, 8, 5, 7, 3]
[1, 0, 2, 4, 6, 8, 5, 7, 3]
[0, 1, 2, 4, 6, 8, 5, 7, 3]
[4, 1, 2, 0, 6, 8, 5, 7, 3]
[4, 1, 2, 5, 6, 8, 0, 7, 3]
[4, 1, 2, 5, 6, 8, 7, 0, 3]
[4, 1, 2, 5, 0, 8, 7, 6, 3]
[4, 1, 2, 5, 8, 0, 7, 6, 3]
[4, 1, 2, 5, 8, 3, 7, 6, 0]
[4, 1, 2, 5, 8, 3, 7, 0, 6]
[4, 1, 2, 5, 0, 3, 7, 8, 6]
[4, 1, 2, 0, 5, 3, 7, 8, 6]
[0, 1, 2, 4, 5, 3, 7, 8, 6]
[1, 0, 2, 4, 5, 3, 7, 8, 6]
[1, 2, 0, 4, 5, 3, 7, 8, 6]
[1, 2, 3, 4, 5, 0, 7, 8, 6]
[1, 2, 3, 4, 5, 6, 7, 8, 0]
